In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df2_WS3T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AS_segment2_workflowS3_has_duration.csv",engine='python')

In [3]:
df2_WS3T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,722111|418134|648088,Assessment Outcome - Successful,Assessment Outcome - Successful,8/12/2018 13:57:55,0 days 0 hours 0 minutes 12 seconds,12.0


In [4]:
df2_WS3T['WRKFLW_TSK_MAP_GROUPED'].nunique()

14

In [5]:
df2_WS3T['WRKFLW_TSK_MAP_DETAILED'].nunique()

15

In [6]:
wd_count = df2_WS3T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Assessment Outcome - Successful', 1116), ('Create Offer', 1113), ('Release Offer', 1105), ('OC - Created - Evidence of course completion', 326), ('Assessment Decision - Qualified', 37), ('Assessment Decision - No Decision', 6), ('OC - Created - English language requirements', 6), ('Put Assessment On-Hold', 6), ('Release Assessment Hold', 6), ('AR - Satisfied - English language proficiency', 3), ('OC - Satisfied - Evidence of course completion', 3), ('Unsuccessful', 3), ('Assessment Outcome - Higher Preference Successful', 1), ('Assessment Outcome - Ineligible / Withdrawn', 1), ('OC - Not Satisfied - Evidence of course completion', 1)]


In [7]:
list = df2_WS3T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df2_WS3T[df2_WS3T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('Unsuccessful', 1705029.0), ('Put Assessment On-Hold', 414526.0), ('OC - Satisfied - Evidence of course completion', 282084.0), ('Assessment Decision - No Decision', 140440.0), ('AR - Satisfied - English language proficiency', 85991.0), ('Create Offer', 16885.0), ('OC - Not Satisfied - Evidence of course completion', 8436.0), ('OC - Created - Evidence of course completion', 7673.0), ('Assessment Decision - Qualified', 1911.0), ('OC - Created - English language requirements', 1905.0), ('Release Assessment Hold', 261.0), ('Assessment Outcome - Successful', 14.0), ('Assessment Outcome - Higher Preference Successful', 9.0), ('Assessment Outcome - Ineligible / Withdrawn', 7.0), ('Release Offer', 0.0)]


In [8]:
numOfWorkflowProcesses = df2_WS3T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df2_WS3T[df2_WS3T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Release Offer']
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df2_WS3T[df2_WS3T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)



In [16]:
s2_t3=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s2_t3['Num of Occurrences per step(descending)'] = list_step_count_per_key
s2_t3['Average time spends per step(descending)'] = list_step_ave_time_per_key
s2_t3['average time for workflow segment(seconds)'] = list_avgtime_per_key

s2_t3['sum time for workflow segment(seconds)'] = list_sumtime_row
s2_t3.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,2,2,0 days 0 hours 0 minutes 12 seconds,0.0 days 0.0 hours 0.0 minutes 12.5 seconds,0.0%,"[(Assessment Outcome - Successful, 2), (Create...","[(Assessment Outcome - Successful, 0.0 days 0....",12,25.0
1,3,753,0 days 2 hours 52 minutes 4 seconds,0.0 days 0.0 hours 2.0 minutes 14.0 seconds,100.0%,"[(Assessment Outcome - Successful, 753), (Crea...","[(Create Offer, 0.0 days 2.0 hours 51.0 minute...",10324,7774016.0
2,4,322,0 days 8 hours 33 minutes 39 seconds,0.0 days 0.0 hours 2.0 minutes 21.5 seconds,99.69%,"[(Assessment Outcome - Successful, 325), (Crea...","[(Create Offer, 0.0 days 6.0 hours 53.0 minute...",30819,9923843.0
3,5,24,2 days 10 hours 25 minutes 16 seconds,0.0 days 0.0 hours 2.0 minutes 48.0 seconds,95.83%,"[(Assessment Outcome - Successful, 27), (Creat...","[(Put Assessment On-Hold, 5.0 days 21.0 hours ...",210316,5047597.0
4,6,6,13 days 7 hours 51 minutes 37 seconds,0.0 days 0.0 hours 47.0 minutes 39.0 seconds,83.33%,"[(Assessment Outcome - Successful, 6), (Create...","[(Put Assessment On-Hold, 8.0 days 9.0 hours 2...",1151497,6908986.0
5,7,1,0 days 0 hours 2 minutes 42 seconds,0.0 days 0.0 hours 2.0 minutes 42.0 seconds,100.0%,"[(OC - Created - Other, 2), (Assessment Outcom...","[(Assessment Decision - Qualified, 0.0 days 0....",162,162.0
6,9,1,5 days 5 hours 11 minutes 43 seconds,5.0 days 5.0 hours 11.0 minutes 43.0 seconds,100.0%,"[(Put Assessment On-Hold, 2), (Release Assessm...","[(Create Offer, 0.0 days 0.0 hours 9.0 minutes...",450703,450703.0
7,10,1,9 days 19 hours 20 minutes 19 seconds,9.0 days 19.0 hours 20.0 minutes 19.0 seconds,100.0%,"[(Assessment Decision - Qualified, 3), (Assess...","[(Assessment Decision - No Decision, 4.0 days ...",847219,847219.0


In [17]:
s2_t3.to_csv('engineering_AS_segment2_workflowS3_table.csv', index=False)






